# mbedオフライン開発環境の構築
岡野さんのページを参考にmbedのオフライン開発環境を構築しました。
- https://developer.mbed.org/users/okano/notebook/setup-mbed-cli-on-mac-os-x-JP/

## 準備
パッケージ管理ソフトHomebrewとXcodeをインストールします。

App Storeを使ってXcodeをインストールした後、最新に更新してください。
mbed cliではXcode Command line Toolsも必要になりますが、mbed cliがインストールを促してくれます。

### Homebrewのインストール
以下のコマンドでHomebewがインストールされます。

```bash
$ ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
```

brewコマンドでpythonとnpmをインストールします。

```bash
$ brew install npm
$ brew install python 
```
pythonのインストールで警告がでるので、以下のコマンドを実行します。
```bash
$ brew link --overwrite python
```

次にgit、Mercurial(hg)とGCCをインストールします。
```bash
$ brew tap PX4/homebrew-px4
$ brew update
$ brew install git hg gcc-arm-none-eabi
```

## mbedオフライン開発環境mbed cliのインストール
mbed cliはpipコマンドを使ってインストールします。これでmbedオフライン開発環境が整いました。
```bash
$ pip install mbed-cli
```


## 動作確認
動作確認のために、Lチカのサンプル(blink)を作成してみます。

```bash
$ mbed new blink --mbedlib
[mbed] Creating new program "blink" (git)
[mbed] Adding library "mbed" from "https://mbed.org/users/mbed_official/code/mbed/builds" at latest revision in the current branch
[mbed] Updating reference "mbed" -> "https://mbed.org/users/mbed_official/code/mbed/builds/tip"
[mbed] Couldn't find build tools in your program. Downloading the mbed 2.0 SDK tools...
[mbed] Auto-installing missing Python modules...
```

main.cppは、以下の様にしました。

In [4]:
%%writefile blink/main.cpp
#include "mbed.h"
 
DigitalOut led1(LED1);
 
int main()
{
    while (1) {
        led1 = !led1;
        wait_ms(500);
    }
}

Writing blink/main.cpp


blinkには、以下のディレクトリとファイルができています。
mbedディレクトリには、必要なライブラリやインクルードファイルがダウンロードされるので、Githubには含めていません。

In [3]:
!ls blink

main.cpp  mbed	mbed.bld  mbed_settings.py  mbed_settings.pyc


### mbed LPC1768で動かす
手元の初期のmbed LPC1768をMacに接続してみます。

mbed cliが正しくmbed LPC1768を認識するかdetectコマンドで確認します。

以下の様に入力すると、LPC1768として認識しました。
```bash
$ mbed detect

[mbed] Detected LPC1768, port /dev/tty.usbmodem1412, mounted /Volumes/MBED
[mbed] Supported toolchains for LPC1768
+---------+-----------+-----------+-----------+-----------+-----------+
| Target  | mbed OS 2 | mbed OS 5 |    ARM    |  GCC_ARM  |    IAR    |
+---------+-----------+-----------+-----------+-----------+-----------+
| LPC1768 | Supported | Supported | Supported | Supported | Supported |
+---------+-----------+-----------+-----------+-----------+-----------+
Supported targets: 1
Supported toolchains: 3
```

それでは、toolchainをGCC_ARM、ターゲットをLPC1768にセットします。
```bash
$ mbed toolchain GCC_ARM
[mbed] GCC_ARM now set as default toolchain in program "blink"
$ mbed target LPC1768
[mbed] LPC1768 now set as default target in program "blink"
```

ライブラリのアップデート（新規の場合ダウンロード）をします。

mbed 2.0 9bcdf88f62b0のバグでLPC1768のdefault targetが見つかりません。その場合には、以下のコメントに従ってください。
- https://github.com/ARMmbed/mbed-cli/issues/391

```bash
$ mbed update
```

コンパイルを実行します。

```bash
$ mbed compile
Building project blink (LPC1768, GCC_ARM)
Scan: .
Scan: mbed
Scan: env
Compile: main.cpp
Link: blink
Elf2Bin: blink
+-----------+-------+-------+------+
| Module    | .text | .data | .bss |
+-----------+-------+-------+------+
| Subtotals | 27529 |  2496 |  632 |
+-----------+-------+-------+------+
Allocated Heap: 2048 bytes
Allocated Stack: 3072 bytes
Total Static RAM memory (data + bss): 3128 bytes
Total RAM memory (data + bss + heap + stack): 8248 bytes
Total Flash memory (text + data + misc): 30025 bytes
Image: ./BUILD/LPC1768/GCC_ARM/blink.bin
```

正常にビルドが完了したので、mbedに書き込みmbedのリセットボタンを押します。LEDが1秒間隔で点滅したら成功です。

```bash
$ cp ./BUILD/LPC1768/GCC_ARM/blink.bin  /Volumes/MBED/
```

## mbedのデバッグ
オフラインの特徴であるデバッグ機能について、順をおって説明します。
 
 